# 2. 鏈的內部運作 (Chains Under the Hood) - Ollama 版本

本範例深入理解 LCEL 鏈的內部實作，展示如何使用 RunnableSequence 手動組合鏈的各個步驟，建立一個文字摘要系統。

## 學習重點
- 了解鏈的底層運作機制
- 學習 RunnableSequence 的使用方法
- 理解 RunnableLambda 的作用
- 對比手動組合與 LCEL 語法的差異


## RunnableSequence 簡介

**RunnableSequence** 是 LangChain 中用來手動組合執行步驟的類別，它讓你可以精確控制鏈的每個環節。

### 基本結構

```python
RunnableSequence(
    first=第一個步驟,      # 必須：開始的步驟
    middle=[中間步驟],     # 可選：中間的多個步驟（列表）
    last=最後一個步驟      # 必須：結束的步驟
)
```

### 核心概念

- **順序執行**: 按照 first → middle → last 的順序依次執行
- **資料流動**: 上一個步驟的輸出會自動成為下一個步驟的輸入
- **完全控制**: 可以在任何位置插入自定義邏輯（如日誌、驗證、轉換等）

### 與 LCEL 的關係

```python
# 這兩種寫法功能相同：
chain = prompt | model | parser          # LCEL 語法（簡潔）
chain = RunnableSequence(                # 手動組合（精細控制）
    first=prompt,
    middle=[model],
    last=parser
)
```

實際上，當你使用 `|` 運算符時，LangChain 內部就是建立 RunnableSequence！

In [11]:
# 導入必要的套件
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableLambda, RunnableSequence
from langchain_google_genai import ChatGoogleGenerativeAI

# 載入環境變數
load_dotenv()

# 建立 Ollama 模型
model = ChatGoogleGenerativeAI(model="gemini-flash-2.5")


In [12]:
# 定義提示模板 - 建立文字摘要系統
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一個專業的文字摘要助手，擅長將長篇文章濃縮成簡潔的重點摘要。"),
        ("human", "請將以下文章摘要成 {max_length} 字以內的重點：\n\n{article}"),
    ]
)




In [13]:
# 手動建立各個可執行步驟 (Runnable)
# 這些步驟對應 LCEL 鏈中的各個元件

# 步驟 1: 格式化提示
# 使用 RunnableLambda 將輸入資料 x 套用到 prompt_template 的 format_prompt 方法
# x 必須是一個 dict，包含 prompt_template 所需的參數（例如 article, max_length）
# 傳出來的資料類型是 `ChatPromptValue`（來自 langchain.prompts.base）
format_prompt = RunnableLambda(lambda x: prompt_template.format_prompt(**x))

# 步驟 2: 呼叫模型
# 這一行使用 RunnableLambda 將前一步產生的 ChatPromptValue 物件（x）轉換為訊息格式（x.to_messages()），
# 並傳遞給 OllamaLLM 的 invoke 方法，取得模型回應。
# 注意：OllamaLLM 的 invoke 方法直接返回字串，而非 AIMessage 物件
invoke_model = RunnableLambda(lambda x: model.invoke(x.to_messages()))

# 步驟 3: 解析輸出
# 因為 OllamaLLM 已經返回字串，所以這裡直接返回即可（不需要 .content）
parse_output = RunnableLambda(lambda x: x)

# 步驟 4: 添加除錯日誌（展示內部運作的價值）
def add_debug_log(step_name):
    """為每個步驟添加除錯日誌，幫助追蹤處理流程"""
    def _log(x):
        print(f"\n{'='*60}")
        print(f"[DEBUG] 步驟: {step_name}")
        print(f"[DEBUG] 輸入類型: {type(x).__name__}")
        if isinstance(x, str):
            print(f"[DEBUG] 內容預覽: {x[:100]}..." if len(x) > 100 else f"[DEBUG] 內容: {x}")
        print(f"{'='*60}")
        return x
    return RunnableLambda(_log)

print("各個步驟已定義完成：")
print("1. format_prompt: 格式化提示模板")
print("2. invoke_model: 呼叫 LLM 模型")
print("3. parse_output: 解析模型輸出")
print("4. add_debug_log: 添加除錯日誌（展示內部控制的優勢）")

各個步驟已定義完成：
1. format_prompt: 格式化提示模板
2. invoke_model: 呼叫 LLM 模型
3. parse_output: 解析模型輸出
4. add_debug_log: 添加除錯日誌（展示內部控制的優勢）


In [14]:
# 使用 RunnableSequence 手動組合鏈，並添加除錯功能
# 這相當於 LCEL 的 prompt_template | model | StrOutputParser()
# 但我們可以在每個步驟之間插入除錯日誌

chain = RunnableSequence(
    first=add_debug_log("開始處理"),
    middle=[
        format_prompt,
        add_debug_log("格式化完成"),
        invoke_model,
        add_debug_log("模型調用完成")
    ],
    last=parse_output
)

print("手動組合的鏈已建立完成（含除錯功能）！")
print("鏈的結構：")
print("  開始 → [DEBUG] → format_prompt → [DEBUG] → invoke_model → [DEBUG] → parse_output")
print("\n💡 這展示了手動組合的優勢：可以在任何步驟之間插入自定義邏輯！")

手動組合的鏈已建立完成（含除錯功能）！
鏈的結構：
  開始 → [DEBUG] → format_prompt → [DEBUG] → invoke_model → [DEBUG] → parse_output

💡 這展示了手動組合的優勢：可以在任何步驟之間插入自定義邏輯！


In [15]:
# 執行手動組合的鏈 - 測試文字摘要系統
sample_article = """
人工智慧（AI）是電腦科學的一個分支，旨在創建能夠執行通常需要人類智能的任務的系統。
這些任務包括學習、推理、問題解決、感知和語言理解。AI 技術已經在許多領域產生重大影響，
包括醫療保健、金融、交通運輸和娛樂。機器學習是 AI 的一個子領域，它使計算機能夠從數據中學習
而無需明確編程。深度學習是機器學習的一個分支，使用人工神經網絡來模擬人腦的工作方式。
隨著技術的進步，AI 有望在未來幾年繼續改變我們的生活和工作方式。
"""

response = chain.invoke({
    "article": sample_article,
    "max_length": 100
})

# 輸出結果
print("=" * 50)
print("文章摘要結果：")
print("=" * 50)
print(response)



[DEBUG] 步驟: 開始處理
[DEBUG] 輸入類型: dict

[DEBUG] 步驟: 格式化完成
[DEBUG] 輸入類型: ChatPromptValue

[DEBUG] 步驟: 模型調用完成
[DEBUG] 輸入類型: str
[DEBUG] 內容預覽: 人工智慧（AI）是電腦科學的一個分支，旨在創建能夠執行通常需要人類智能的任務。AI技術已經產生重大影響，在醫療、金融、交通和娛樂等領域。機器學習是一種 AI 技術，使用數據來學習而無需明確編程。深度學...
文章摘要結果：
人工智慧（AI）是電腦科學的一個分支，旨在創建能夠執行通常需要人類智能的任務。AI技術已經產生重大影響，在醫療、金融、交通和娛樂等領域。機器學習是一種 AI 技術，使用數據來學習而無需明確編程。深度學習是該技術的一個分支，使用人工神經網絡模擬人腦的工作方式。


### 理解數據流
```
chain.invoke({"article": "...", "max_length": 100})
    ↓
開始處理: dict
    ↓
format_prompt: dict → ChatPromptValue
    ↓
格式化完成: ChatPromptValue
    ↓
invoke_model: ChatPromptValue → str
    ↓
模型調用完成: str
    ↓
parse_output: str → str
```

## 💡 重點說明

### 手動組合 vs LCEL 語法

**手動組合 (RunnableSequence)**:
```python
chain = RunnableSequence(
    first=add_debug_log("開始"),
    middle=[format_prompt, add_debug_log("格式化"), invoke_model],
    last=parse_output
)
```

**LCEL 語法**:
```python
chain = prompt_template | model | StrOutputParser()
```

### 優缺點比較

**手動組合的優點**:
- ✅ **精細控制**: 可以在任何步驟之間插入自定義邏輯
- ✅ **除錯友好**: 容易添加日誌記錄和錯誤處理
- ✅ **學習價值**: 清楚看到每個步驟的實作細節
- ✅ **效能監控**: 可以追蹤每個步驟的執行時間

**LCEL 語法的優點**:
- ✅ **程式碼簡潔**: 一行就能完成鏈的定義
- ✅ **自動功能**: 自動具備串流、批次處理等功能
- ✅ **推薦做法**: LangChain 官方推薦的現代化方法
- ✅ **易於維護**: 結構清晰，容易理解

### 🎯 實際應用建議

**使用手動組合的場景**:
1. **開發階段**: 需要詳細的除錯資訊
2. **效能優化**: 需要監控每個步驟的執行時間
3. **錯誤處理**: 需要對特定步驟進行錯誤捕獲
4. **學習理解**: 想要深入理解 LangChain 的內部機制

**使用 LCEL 的場景**:
1. **生產環境**: 程式碼經過充分測試，不需要詳細除錯
2. **快速開發**: 需要快速建立原型
3. **簡單流程**: 不需要複雜的中間處理
4. **團隊協作**: 程式碼需要清晰易讀

## 🔧 實際應用場景

- **新聞摘要**: 自動生成新聞重點，並記錄處理時間
- **學術論文**: 提取研究重點，追蹤每個處理步驟
- **會議記錄**: 整理會議要點，添加品質檢查
- **長文閱讀**: 快速了解文章內容，監控處理效能

## 📊 除錯輸出範例

從上面的執行結果可以看到，我們能夠追蹤：
- 每個步驟的輸入資料類型
- 處理過程中的資料轉換
- 中間結果的預覽

這些資訊在開發和除錯時非常有價值！